Libraries

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.stats import norm
import torch

Generate the samples from truth

For now we use: 
B_0 = 0,
A_0 = 0,
var = 1
and seed 508

In [2]:
print('Enter the parameters for the truth.')
print('B_0 =')
B_0 = float(input())
print('A_0 =')
A_0 = float(input())
print('var =')
var = float(input())

print('Enter the random seed: ', end='')
np.random.seed(int(input()))

S=np.random.normal(B_0,A_0*A_0+var,500)

Enter the parameters for the truth.
B_0 =


 0


A_0 =


 0


var =


 1


Enter the random seed: 

 508


Define p(x|z) and q(z|x)

In [3]:
def p_dist(S,z,A,B):
    prob = [norm.pdf(x,A*z+B,1) for x in S]
    return np.prod(prob)

def q_dist(z,S,C,D):
    prob = [norm.pdf(z,C*x+D,1) for x in S]
    return np.prod(prob)

Define E_{q(z|x)}(p(x|z))

In [13]:
def E_q(A,B,C,D,S):
    def formula(z):
        if p_dist(S,z,A,B)==0:
            return 0
        else:
            return np.log(p_dist(S,z,A,B))*q_dist(z,S,C,D)

    return quad(formula,-np.inf,np.inf)[0];

Define the kl divergence, KL( q(z|x) || p(z) )

In [ ]:
def kl_divergence(self, z, mu, std):
    # --------------------------
    # Monte carlo KL divergence
    # --------------------------
    # 1. define the first two probabilities (in this case Normal for both)
    p = torch.distributions.Normal(torch.zeros_like(mu), torch.ones_like(std))
    q = torch.distributions.Normal(mu, std)

    # 2. get the probabilities from the equation
    log_qzx = q.log_prob(z)
    log_pz = p.log_prob(z)

    # kl
    kl = (log_qzx - log_pz)
    
    # sum over last dim to go from single dim distribution to multi-dim
    kl = kl.sum(-1)
    return kl

In [5]:
def kl_divergence(A,B,C,D,S):
    def formula(z):
        if (norm.pdf(z)==0 or q_dist(z,S,C,D)==0):
            return 0
        else:
            return q_dist(z,S,C,D)*np.log(q_dist(z,S,C,D)/norm.pdf(z,0,1))

    return quad(formula,-np.inf,np.inf)[0]

Define the ELBO

In [6]:
def L(A,B,C,D,S):
    return E_q(A,B,C,D,S) - kl_divergence(A,B,C,D,S)

In [20]:
print(E_q(1,2,3,4,[0]))
print(L(1,2,3,4,[0]))
print(q_dist(0,S,3,4))

-19.41893853320468
-27.41893853320468
0.0
